In [1]:
%pip install requests beautifulsoup4 lxml pandas




Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup

print("Libraries are working!")


Libraries are working!


In [3]:
from bs4 import BeautifulSoup

html = "<html><body><h1>Test</h1></body></html>"
soup = BeautifulSoup(html, "html.parser")
print(soup.h1.text)


Test


In [4]:
# for i in range(2,12):
#     link = "https://www.flipkart.com/search?q=iphone+under+50000&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_18_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_18_na_na_na&as-pos=2&as-type=HISTORY&suggestionId=iphone+under+50000%7CMobiles&requestId=71dc07b0-11fb-4a34-9aa6-0b0b22b7d16b&page="+ str(i)
#     url = requests.get(link)
#     print(url) 


#     soup = BeautifulSoup(url.text, "html.parser")
#     box= soup.find("div", class_="DOjaWF gdgoEp")

#     np = box.find("a", class_="_9QVEpD").get("href")
#     cnp= "https://www.flipkart.com/" + np 
#     print(cnp)
#     Product_name=[]
#     names = box.find_all("div", class_="KzDlHZ")
#     for i in names :
#         name = i.text
#         Product_name.append(name)
#     # print(Product_name)
#     print(len(Product_name))
#     Prices=[] 


#     prices = box.find_all("div", class_="Nx9bqj _4b5DiR")
#     for i in prices :
#         price = i.text
#         Prices.append(price)
#         len(prices)
#     # print(Prices)

#     Description=[] 


#     desc = soup.find_all("ul", class_="G4BRas")
#     for i in desc :
#         Descriptions = i.text
#         Description.append(Descriptions)
#     # print(Description)
#     print(len(Description))


#     Ratings=[] 


#     rate = box.find_all("div", class_="XQDdHH")
#     for i in rate :
#         ratings = i.text
#         Ratings.append(ratings)
#     print(len(Ratings))
#     # print(Ratings)

#     product_links = []
#     products = box.find_all("a", class_="CGtC98")  # This is usually the product anchor tag class

#     for p in products:
#         product_links.append("https://www.flipkart.com" + p.get("href"))

#     print(len(product_links))


In [5]:
import requests
from bs4 import BeautifulSoup
import time
import os
import json

all_products = []

def save_data(data, filename="flipkart_iphones_with_reviews_qna2.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"[INFO] Data saved. Total products saved so far: {len(data)}\n")
def download_image(image_url, product_name):
    if not image_url:
        return None
    filename = "".join(c for c in product_name if c.isalnum() or c in (' ', '_')).rstrip()
    filepath = f"flipkart_images/{filename}.jpg"
    try:
        os.makedirs("flipkart_images", exist_ok=True)
        img_data = requests.get(image_url).content
        with open(filepath, 'wb') as handler:
            handler.write(img_data)
        print(f"  - Image downloaded: {filepath}")
        return filepath
    except Exception as e:
        print(f"  - Failed to download image: {e}")
        return None

for page in range(2, 12):
    print(f"[PAGE] Starting scraping page {page}...")
    url = f"https://www.flipkart.com/search?q=iphone+under+50000&sid=tyy%2C4io&page={page}"
    try:
        response = requests.get(url)
        print(f"[REQUEST] Page {page} fetched successfully (Status code: {response.status_code})")
        soup = BeautifulSoup(response.text, "html.parser")
    except Exception as e:
        print(f"[ERROR] Failed to load page {page}: {e}")
        continue

    container = soup.find("div", class_="DOjaWF gdgoEp")
    if not container:
        print(f"[WARNING] No main container found on page {page}")
        continue
    else:
        print(f"[INFO] Main container found on page {page}")

    product_cards = container.find_all("div", class_="tUxRFH")
    print(f"[INFO] Found {len(product_cards)} product cards on page {page}")

    for idx, card in enumerate(product_cards, start=1):
        print(f"[PRODUCT {idx}] Extracting product details...")
        try:
            name = card.find("div", class_="KzDlHZ").text.strip()
            print(f"  - Name: {name}")
        except:
            name = None
            print("  - Name: Not found")

        try:
            price = card.find("div", class_="Nx9bqj _4b5DiR").text.strip()
            print(f"  - Price: {price}")
        except:
            price = None
            print("  - Price: Not found")

        try:
            description = card.find("ul", class_="G4BRas").text.strip()
            print(f"  - Description: {description[:60]}...")  # print first 60 chars only
        except:
            description = None
            print("  - Description: Not found")

        try:
            rating = card.find("div", class_="XQDdHH").text.strip()
            print(f"  - Rating: {rating}")
        except:
            rating = None
            print("  - Rating: Not found")

        try:
            link_tag = card.find("a", class_="CGtC98")
            product_link = "https://www.flipkart.com" + link_tag["href"] if link_tag else None
            print(f"  - Product link found: {product_link}")
        except:
            product_link = None
            print("  - Product link: Not found")
        try:
            img_tag = card.find("img")
            image_url = img_tag["src"] if img_tag else None
            print(f"  - Image URL: {image_url}")
        except:
            image_url = None
            print("  - Image URL: Not found")

        
        image_path = download_image(image_url, name or f"product_{idx}_page_{page}")


        reviews = []
        qna = []

        if product_link:
            print(f"  [VISIT] Visiting product page for reviews and Q&A...")
            try:
                product_response = requests.get(product_link)
                print(f"  [REQUEST] Product page fetched (Status code: {product_response.status_code})")
                product_soup = BeautifulSoup(product_response.text, "html.parser")

                review_divs = product_soup.find_all("div", class_="col EPCmJX")
                print(f"  [INFO] Found {len(review_divs)} review divs")
                for r in review_divs:
                    reviews.append(r.get_text(strip=True))

                qna_divs = product_soup.find_all("div", class_="BZMA+t")
                print(f"  [INFO] Found {len(qna_divs)} Q&A divs")
                for q in qna_divs:
                    qna.append(q.get_text(strip=True))

            except Exception as e:
                print(f"  [ERROR] Failed to get data from product page: {e}")

            print("  [WAIT] Sleeping for 2 seconds to avoid rate limits...\n")
            time.sleep(2)
        else:
            print("  [SKIP] No product link available; skipping reviews and Q&A.\n")

        category = "Smartphone"

# Convert description to specifications (basic split by newlines or bullets)
        specifications = []
        if description:
    # Split by bullets or newlines and clean entries
            raw_specs = description.split("•") if "•" in description else description.split("\n")
            specifications = [spec.strip() for spec in raw_specs if spec.strip()]


        product_info = {
            "name": name,
            "price": price,
            "description": description,
            "specifications": specifications,
            "rating": rating,
            "category": category,
            "image_url": image_url,
            "image_path": image_path,
            "link": product_link,
            "reviews": reviews,
            "qna": qna
        }

        all_products.append(product_info)

        if len(all_products) % 10 == 0:
            print(f"[SAVE] Saving data after scraping {len(all_products)} products...")
            save_data(all_products)

print("[FINAL] Scraping completed. Saving all data...")
save_data(all_products)

print(f"[DONE] Total products scraped: {len(all_products)}")


[PAGE] Starting scraping page 2...
[REQUEST] Page 2 fetched successfully (Status code: 200)
[INFO] Main container found on page 2
[INFO] Found 24 product cards on page 2
[PRODUCT 1] Extracting product details...
  - Name: Apple iPhone 8 (Gold, 64 GB)
  - Price: ₹39,900
  - Description: 64 GB ROM11.94 cm (4.7 inch) Retina HD Display12MP Rear Came...
  - Rating: 4.5
  - Product link found: https://www.flipkart.com/apple-iphone-8-gold-64-gb/p/itmexrgvcbsga3zz?pid=MOBEXRGVZWHDW2RQ&lid=LSTMOBEXRGVZWHDW2RQYDS1SL&marketplace=FLIPKART&q=iphone+under+50000&store=tyy%2F4io&srno=s_2_25&otracker=search&fm=organic&iid=18e0e683-ad9c-4b8f-ad5f-a52ce70820a3.MOBEXRGVZWHDW2RQ.SEARCH&ppt=None&ppn=None&ssid=ine8sw9txc0000001748322262610&qH=26ef41cdd426edbb
  - Image URL: https://rukminim2.flixcart.com/image/312/312/j7ksia80/mobile/2/r/q/apple-iphone-8-mq6m2hn-a-original-imaexsbzyvydzgzw.jpeg?q=70
  - Image downloaded: flipkart_images/Apple iPhone 8 Gold 64 GB.jpg
  [VISIT] Visiting product page for review